In [3]:
import open3d as o3d
import trimesh

In [2]:
input_points_path = "./open3d/quicktest_undc_noisypc.obj"
input_mesh trimesh.load(input_points_path)
# point_cloud_data = o3d.io.read_point_cloud(input_points_path)

[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for ./open3d/quicktest_undc_noisypc.obj (format: auto).
